In [9]:
import cv2

In [10]:
# อ่านภาพ
image = cv2.imread('D:\\WORK\\Windows\\CODE_BME\\PROJECT_G6PD\\G6PD_DATA_SET\\1-5.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [11]:
# ใช้ Threshold
_, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)

In [12]:
# ค้นหา Contours
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [13]:
# วนซ้ำตรวจสอบ Contours ที่เป็นวงกลม
for contour in contours:
    (x, y), radius = cv2.minEnclosingCircle(contour)
    if radius > 10:  # กรองขนาดรัศมี
        mask = np.zeros_like(image)
        cv2.circle(mask, (int(x), int(y)), int(radius), (255, 255, 255), -1)
        result = cv2.bitwise_and(image, mask)
        cropped = result[int(y - radius):int(y + radius), int(x - radius):int(x + radius)]

        # แสดงผล
        cv2.imshow('Cropped Circle', cropped)
        cv2.waitKey(0)

cv2.destroyAllWindows()